## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config_copy2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Upernavik,GL,72.7868,-56.1549,43.97,61,100,10.11,overcast clouds
1,1,Bambous Virieux,MU,-20.3428,57.7575,75.22,67,18,13.31,few clouds
2,2,Hasaki,JP,35.7333,140.8333,77.29,83,100,29.59,overcast clouds
3,3,Dikson,RU,73.5069,80.5464,44.42,86,71,2.75,broken clouds
4,4,San Patricio,US,28.0170,-97.5169,81.84,96,55,5.03,broken clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                        (city_data_df["Max Temp"] > min_temp)]
clean_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Bambous Virieux,MU,-20.3428,57.7575,75.22,67,18,13.31,few clouds
2,2,Hasaki,JP,35.7333,140.8333,77.29,83,100,29.59,overcast clouds
4,4,San Patricio,US,28.0170,-97.5169,81.84,96,55,5.03,broken clouds
15,15,Salalah,OM,17.0151,54.0924,82.49,78,75,16.11,broken clouds
19,19,Atuona,PF,-9.8000,-139.0333,77.41,73,38,23.60,scattered clouds


In [7]:
# 4a. Determine if there are any empty rows.
clean_df.count()

Unnamed: 0             188
City                   188
Country                185
Lat                    188
Lng                    188
Max Temp               188
Humidity               188
Cloudiness             188
Wind Speed             188
Current Description    188
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = clean_df.dropna()

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng", "Current Description"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
1,Bambous Virieux,MU,75.22,-20.3428,57.7575,few clouds,
2,Hasaki,JP,77.29,35.7333,140.8333,overcast clouds,
4,San Patricio,US,81.84,28.0170,-97.5169,broken clouds,
15,Salalah,OM,82.49,17.0151,54.0924,broken clouds,
19,Atuona,PF,77.41,-9.8000,-139.0333,scattered clouds,
21,Meulaboh,ID,83.25,4.1363,96.1285,overcast clouds,
22,Vaini,TO,75.31,-21.2000,-175.2000,overcast clouds,
23,Chhabra,IN,84.60,24.6667,76.8333,overcast clouds,
26,Kavaratti,IN,81.77,10.5669,72.6420,overcast clouds,
27,Kapaa,US,80.92,22.0752,-159.3190,light rain,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()
# good

City                   185
Country                185
Max Temp               185
Lat                    185
Lng                    185
Current Description    185
Hotel Name             185
dtype: int64

In [10]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
1,Bambous Virieux,MU,75.22,-20.3428,57.7575,few clouds,Casa Tia Villa
2,Hasaki,JP,77.29,35.7333,140.8333,overcast clouds,Hotel Sunrise Choshi
4,San Patricio,US,81.84,28.0170,-97.5169,broken clouds,"Motel 6 Sinton, TX"
15,Salalah,OM,82.49,17.0151,54.0924,broken clouds,HAMDAN PLAZA HOTEL SALALAH
19,Atuona,PF,77.41,-9.8000,-139.0333,scattered clouds,Villa Enata


In [13]:
# 8a/b. Create the output File (CSV)
hotel_df.to_csv('WeatherPy_vacation.csv')


In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>    
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# 11a/b Add a marker layer for each city to the map. 

locations = hotel_df[["Lat","Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))